In [ ]:
import os
import sys
from datetime import datetime
sys.path.insert(0, "../../..")
sys.path.insert(0, "../../../../acquire")

from HUGS.Processing import search
from HUGS.Client import Process, Search, Retrieve
from Acquire.ObjectStore import datetime_to_string
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

import ipyleaflet as ipl
import ipywidgets as ipw

from pathlib import Path

# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

Search for some EUROCOM data

In [ ]:
from HUGS.Client import Search
from HUGS.Util import get_datapath
import json

In [ ]:
base_url= "https://hugs.acquire-aaai.com/t"
search = Search(service_url=base_url)

In [ ]:
acrg_json = get_datapath(filename="acrg_site_info.json")
with open(acrg_json, "r") as f:
    acrg_sites = json.load(f)

Panel 1 - NOAA with EDGAR

In [ ]:

import matplotlib.pyplot as plt
from cartopy.feature import BORDERS
import cartopy.crs as ccrs
import matplotlib.cm as cm
import xarray as xr
import numpy as np
import ipyleaflet as ipl
from random import uniform

# filepath = "/home/gar/Documents/Devel/hugs/user/notebooks/openghg/ch4-anthro_GLOBAL_2012.nc"

# Here try and work  out projection issues

filepath = "/home/home/gar/Documents/Devel/RSE/hugs/raw_data/ch4-anthro_GLOBAL_2012.nc"

ds = xr.open_dataset(filepath)

domain = "EUROPE"

fig = plt.figure()
# ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax = fig.add_subplot(111, projection=ccrs.Mercator())

ax.coastlines(color="0.2")
ax.add_feature(BORDERS, edgecolor="0.5")

fp_name = "flux"
lon_name = "lon"
lat_name = "lat"

cmap = cm.get_cmap("magma")
levels = np.linspace(np.percentile(ds[fp_name].values, 5), np.percentile(ds[fp_name].values, 95), 20)

long_values = ds[fp_name][lon_name].values
# long_values = long_values[:1800]
lat_values = ds[fp_name][lat_name].values
zero_values = ds[fp_name][:, :, 0].values

# NE corner
max_long = ds[fp_name][lon_name].max()
max_lat = ds[fp_name][lat_name].max()

# SW corner
min_long = ds[fp_name][lon_name].min()
min_lat =  ds[fp_name][lat_name].min()

# print(ds.keys())

# print(ds)


# locations = [[uniform(-80, 80), uniform(-180, 180), uniform(0, 1000)] for i in range(1000)]

# print(type(long_values))

# print(len(long_values))
# print(len(lat_values))
# print(len(levels))

# print(zero_values[15])


# locations = [[a,b,uniform(0,100)] for a,b in zip(long_values, lat_values)]

# for x in range(5):
#     print(lat_values[x], long_values[x], zero_values[x])


# locations = [long_values.tolist(), lat_values.tolist(), zero_values.tolist()]

# print(locations[1])

# m = ipl.Map(center=[0,0], zoom=2)
# heat = ipl.Heatmap(locations=locations, radius=20)

# m.add_layer(heat)

# m

#     plt.imshow(long_values, lat_values, zero_values)

# ax.contourf(long_values, lat_values, zero_values,cmap=cm.get_cmap("magma"), levels=levels)

# fig.savefig("emissions_edgar.png", dpi=300, bbox_inches="tight")

In [ ]:
max_long, max_lat

In [ ]:
min_long, min_lat

In [ ]:
emissions_file = "/home/gar/Documents/Devel/hugs/user/notebooks/openghg/ch4-anthro_GLOBAL_2012.nc"

plot_emissions(filepath=emissions_file)

In [ ]:
import ipyleaflet as ipl
    
center = [38, 0]
zoom = 0
noaa_map = ipl.Map(center=center, zoom=zoom)

# 179.95, dtype=float32), <xarray.DataArray 'lat' ()>
#  array(89.95,

edgar_image_path = "emissions_edgar.png"

edgar_layer = ipl.ImageOverlay(url=edgar_image_path, bounds=((-80,-180),(90,180)))

noaa_map.add_layer(edgar_layer)
                               


no_lat_long = []
    
for site in acrg_sites:
    network_key = list(acrg_sites[site].keys())[0]
    
    if network_key == "NOAA":
        site_data = acrg_sites[site][network_key]
        
        try:
            lat = acrg_sites[site][network_key]["latitude"]
            long = acrg_sites[site][network_key]["longitude"]

            marker = ipl.Marker(location=(lat, long), draggable=False)
            marker.popup = ipw.HTML(value=str(site))
            noaa_map.add_layer(marker)
        except:
            no_lat_long.append(site)
            
        
noaa_map
        
        
        


Panel 2 - EUROCOM with footprint

In [ ]:
icos_json = get_datapath(filename="icos_eurocom_sites.json")

center = [54.2361, -4.548]
zoom = 3
site_map = ipl.Map(center=center, zoom=zoom)

with open(icos_json, "r") as f:
    icos_sites = json.load(f)

icos_markers = []

for site in icos_sites:
    lat = icos_sites[site]["latitude"]
    long = icos_sites[site]["longitude"]
    site_name = icos_sites[site]["long_name"]
    
    marker = ipl.Marker(location=(lat, long), draggable=False)
    marker.popup = ipw.HTML(value=f"Site code: {site}<br>Site name: {site_name}")
    site_map.add_layer(marker)

site_map

Panel 3 - London with Canopy and Footprint

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles


center = [51.506815, -0.127710]
zoom = 12
map_london = ipl.Map(center=center, zoom=zoom)

dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.Positron)
map_london.add_layer(dark_matter_layer)

lghg_sites = get_datapath("lghg_sites.json")

# Fix legend here
marker_legend = ipw.HTML(value="<img src='marker-icon-green.png'>: Future<br>Blue marker: Current")
marker_control = ipl.WidgetControl(widget=marker_legend, position="topright")

map_london.add_control(marker_control)

with open(lghg_sites, "r") as f:
    lghg_data = json.load(f)
    

for site in lghg_data["current"]:
    curr_site = lghg_data["current"][site]
    
    lat = curr_site["latitude"]
    long = curr_site["longitude"]
    site_name = curr_site["long_name"]
    
    marker = ipl.Marker(location=(lat, long), draggable=False)
    marker.popup = ipw.HTML(value=f"Site code: {site}<br>Site name: {site_name}")
    map_london.add_layer(marker)
    
for site in lghg_data["future"]:
    fut_site = lghg_data["future"][site]
    
    lat = fut_site["latitude"]
    long = fut_site["longitude"]
    site_name = fut_site["long_name"]
    
    # Here we want a green icon
    icon = ipl.Icon(icon_url='marker-icon-green.png', icon_size=[25, 40], icon_anchor=[12,15])
    
    marker = ipl.Marker(location=(lat, long), draggable=False, icon=icon)
    marker.popup = ipw.HTML(value=f"Site code: {site}<br>Site name: {site_name}")
    map_london.add_layer(marker)
    
    
# Here we can load in just some canopy tiles for Central London
# see https://data.london.gov.uk/dataset/curio-canopy and tile-grid-lookup.zip for the tiles

# with open('/home/home/gar/Documents/Devel/RSE/hugs/canopy_data/gla-hexagon-grid-canopy-cover.geojson', 'r') as f:
#   data = json.load(f)

# geo_json = ipl.GeoJSON(data=data)
# agage_map.add_layer(geo_json)

agage = get_datapath(filename="icos_eurocom_sites.json")

map_london